In [1]:
# ~~~~~~~~~~~~~~ Libraries
import sys, os
import mne # Python package for processing and analyzing electrophysiological data
import numpy as np
from glob import glob # look for all the pathnames matching a specified pattern according to the rules
import matplotlib.pyplot as plt
from mne.preprocessing import ICA # ICA (Independent Component Analysis) algorithm, which is for artifact removal
from autoreject import AutoReject # Python package for automatically rejecting bad epochs in EEG/MEG data
import json
import owncloud
import pandas as pd
import braindecode
import torch
import re
import pickle
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict, cross_validate, StratifiedKFold
from sklearn.utils import compute_class_weight

# Deep learning
from braindecode.models import EEGNetv4
from braindecode.preprocessing import exponential_moving_standardize
from braindecode.util import set_random_seeds
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split
from braindecode import EEGClassifier
# ~~~~~~~~~~~~~~ Libraries ~~~~~~~~~~~~~~

In [2]:
# ~~~~~~~~~~~~~~ Parameters
group = 'baby'
modality = 'visual' # 'visual' or 'audio'
subject = 'sub-01'
analysis = 'condition-wise' # 'all' or 'condition-wise'

# EEGNET parameters
# Load the JSON file
code_directory = os.path.dirname("/u/kazma/MINT/code/1st-level")
sys.path.append(code_directory) 
os.chdir(os.path.dirname("/u/kazma/MINT/code/1st-level"))
print(os.getcwd())
with open("config.json") as f: # import variables from config.json
    config = json.load(f) 
globals().update(config)


# Print out each parameter
print(f"{modality} data of {group}: {subject} is processed")
print("EEGNET parameters:")
print(f"====== CV: {EEGNET_CV}")
print(f"====== Max epochs: {EEGNET_MAX_EPOCHS}")
print(f"====== Batch size: {EEGNET_BATCH_SIZE}")

/viper/u/kazma/MINT/code
visual data of baby: sub-01 is processed
EEGNET parameters:
====== CV: 10
====== Max epochs: 200
====== Batch size: 16


In [3]:

# ~~~~~~~~~~~~~~ read 1 session
each_sub_path = f"{COMPUTE_DIR}/data/{group}/interim/{modality}/{subject}"
each_sub_folders = [f for f in os.listdir(each_sub_path) if os.path.isdir(os.path.join(each_sub_path, f))]
each_sub_folders_sorted = sorted(each_sub_folders, key=lambda x: int(re.search(r'\d+', x).group()))

sub_filename = os.path.join(each_sub_path, 'epochs-epo.fif') 
epochs = mne.read_epochs(sub_filename, preload=True)

# ~~~~~~~~~~~~~~ read 1 session ~~~~~~~~~~~~~~



Reading /ptmp/kazma/DATA-MINT/data/baby/interim/visual/sub-01/epochs-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...    1000.00 ms
        0 CTF compensation matrices available
Not setting metadata
847 matching events found
No baseline correction applied
0 projection items activated


In [4]:
# ~~~~~~~~~~~~~~ read 1 session csv file
each_sub_path = f"{COMPUTE_DIR}/data/{group}/raw/{modality}/{subject}"

csv_file = f"{subject}_ses-01_{modality}.csv"
sub_filename = os.path.join(each_sub_path, csv_file) 
csv_dfs = pd.read_csv(sub_filename)
# ~~~~~~~~~~~~~~ read 1 session csv file ~~~~~~~~~~~~~~


# ~~~~~~~~~~~~~~ Obtain target conditions' trials
csv_dfs['Extracted'] = csv_dfs['condition'].str.split('_').str[-1]
csv_dfs = csv_dfs[csv_dfs["Attention"]!= 0].reset_index(drop=True)

csv_idx = csv_dfs[['condition', 'Extracted', 'Attention']]
condition_lists = csv_idx['Extracted'].unique()





In [ ]:
if analysis == 'all':
    cropped_epochs = epochs.copy().crop(tmin=EEGNET_MIN_TIME, tmax=EEGNET_MAX_TIME)

    # Get the info about the cropped data
    conditions = list(cropped_epochs.event_id.keys()) # list of conditions
    n_conditions = len(conditions) # number of conditions
    n_trials = len(cropped_epochs) # number of trials
    n_samples = cropped_epochs.get_data().shape[2]
    n_channels = cropped_epochs.get_data().shape[1]
    min_time = cropped_epochs.times[0]*1000   # First time point in milli seconds
    max_time = cropped_epochs.times[-1]*1000    # Last time point in milli seconds


    print("=====================================================")
    print("=====================================================")
    print("=====================================================")

    print(f" Condition lists: {conditions}")
    print(f" Total trials: {n_trials}")
    print(f" Time points: {n_samples}")
    print(f" Time window (ms): {min_time} - {max_time}")


    # check GPU availability
    cuda = torch.cuda.is_available()  # Check if a GPU is available
    device = "cuda" if cuda else "cpu"  # Use "cuda" if available, otherwise fallback to "cpu"

    if cuda:
        torch.backends.cudnn.benchmark = True  # Enable cuDNN auto-tuning for performance


    # Set random seeds
    # PURPOSE: reproducibility across runs, random initializations (e.g., model weights) yield consistant results
    seed = 20200220
    set_random_seeds(seed=seed, cuda=cuda)


    # dictionaries for decoding accuracy
    pairwise_decoding_accuracies = {}
    # dictionaries for decoding accuracy of standard deviation
    pairwise_decoding_accuracies_std = {}
    # dictionaries for estimators
    pairwise_decoding_estimators = {}



    for i in range(n_conditions):
        for j in range(i + 1, n_conditions):

            cond1 = conditions[i]
            cond2 = conditions[j]

            filtered_epochs = cropped_epochs[cond1,cond2]

            # get EEG data
            X = filtered_epochs.get_data(picks=mne.pick_types(filtered_epochs.info, eeg=True, eog=False, exclude='bads'))

            # get labels (=numerosity)
            unique_labels = np.unique(filtered_epochs.events[:,-1])
            label_0, label_1 = unique_labels[0], unique_labels[1]  # Assign the first label to 0 and the second to 1

            y = np.where(filtered_epochs.events[:, -1] == label_0, 0, 1)

            counts = np.bincount(y)
            print(f"{cond1} vs {cond2}")
            print(f"{counts[0]} vs. {counts[1]} trials")

            skfold = StratifiedKFold(n_splits=EEGNET_CV, shuffle=True, random_state=23)

            # exp. moving std. for each trial
            for s in range(X.shape[0]):
                X[s,:,:] = exponential_moving_standardize(X[s,:,:], factor_new=0.001, init_block_size=None, eps=1e-4)

            class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)


            # create the model
            net = EEGClassifier(
                "EEGNetv4", 
                            module__n_chans=n_channels, # Number of EEG channels
                            module__n_outputs=2,               # Number of outputs of the model. This is the number of classes in the case of classification.
                            module__n_times=n_samples,         # Number of time samples of the input window.
                            module__final_conv_length='auto',  # Length of the final convolution layer. If "auto", it is set based on the n_times.
                            module__pool_mode='mean',          # Pooling method to use in pooling layers
                            module__F1=8,                      # Number of temporal filters in the first convolutional layer.
                            module__D=2,                       # Depth multiplier for the depthwise convolution.
                            module__F2=16,                     # Number of pointwise filters in the separable convolution. Usually set to ``F1 * D``.
                            module__kernel_length=64,         # Length of the temporal convolution kernel. Usally sampling rate / 2 = 500/2 = 250
                            module__third_kernel_size=(8, 4), 
                            module__drop_prob=0.25,            # Dropout probability after the second conv block and before the last layer. 0.5 for within-subject classification, 0.25 in cross-subject clasification
                            module__chs_info=None,             # (list of dict) – Information about each individual EEG channel. This should be filled with info["chs"]. Refer to mne.Info for more details.
                            module__input_window_seconds=None, # Length of the input window in seconds.
                            module__sfreq=SFREQ,
                            max_epochs=EEGNET_MAX_EPOCHS,
                            batch_size=EEGNET_BATCH_SIZE,
                            train_split=None,
                            criterion__weight=torch.Tensor(class_weights).to('cpu'), # class weight
            )
            
            cvs = cross_validate(net, 
                                X, 
                                y, 
                                scoring="balanced_accuracy", # for balanced classes, this corresponds to accuracy,
                                # chance level might be 0 (adjusted = False), or 0.X (adjusted = True)
                                cv=skfold, 
                                n_jobs=-1, # only 1 to avoid overload in parallel jobs, in non-par jobs it could be -1
                                return_estimator=True, # if you need the model to estimate on another test set
                                return_train_score=False,
                                )
                        
            pairwise_decoding_accuracies[(cond1,cond2)] = np.mean(cvs['test_score'])
            pairwise_decoding_accuracies_std[(cond1,cond2)] = np.std(cvs['test_score'])
            pairwise_decoding_estimators[(cond1,cond2)] = cvs['estimator']


    # ~~~~~~~~~~~~~~~~ Save the decoding accuracy
    save_folder = f"{COMPUTE_DIR}/data/{group}/processed/{modality}/{subject}"

    filename = f"EEGNet_accuracy_pairwise_{analysis}_timewindow350.pkl"
    save_path = os.path.join(save_folder, filename) #  a pickle file
    with open(save_path, "wb") as f:
        pickle.dump(pairwise_decoding_accuracies, f)
    print(f"{subject}: saved in {filename}")
    # ~~~~~~~~~~~~~~~~ Save the decoding accuracy ~~~~~~~~~~~~~~~~

    # ~~~~~~~~~~~~~~~~ Save the decoding accuracy standard deviation
    filename = f"EEGNet_std_pairwise_{analysis}_timewindow350.pkl"
    save_path = os.path.join(save_folder, filename) #  a pickle file
    with open(save_path, "wb") as f:
        pickle.dump(pairwise_decoding_accuracies_std, f)
    print(f"{subject}: saved in {filename}")
    # ~~~~~~~~~~~~~~~~ Save the decoding accuracy standard deviation ~~~~~~~~~~~~~~~~


    # ~~~~~~~~~~~~~~~~ Save the EEGNet estimation
    filename = f"EEGNet_estimator_pairwise_{analysis}_timewindow350.pkl"
    save_path = os.path.join(save_folder, filename) #  a pickle file
    with open(save_path, "wb") as f:
        pickle.dump(pairwise_decoding_estimators, f)
    print(f"{subject}: saved in {filename}")
    # ~~~~~~~~~~~~~~~~ Save the EEGNet estimation ~~~~~~~~~~~~~~~~


else:
    for target_condition in condition_lists:

        idx_to_keep = csv_dfs.index[csv_dfs['Extracted'] == target_condition].tolist()

        selected_epochs = epochs[idx_to_keep]
        print(f"Selected trials: {len(selected_epochs)}")
        print(f"Target condition: {target_condition}") 
        # ~~~~~~~~~~~~~~ Obtain target conditions' trials ~~~~~~~~~~~~~~
        # Crop epochs to the desired time range
        cropped_epochs = selected_epochs.copy().crop(tmin=EEGNET_MIN_TIME, tmax=EEGNET_MAX_TIME)

        # Get the info about the cropped data
        conditions = list(cropped_epochs.event_id.keys()) # list of conditions
        n_conditions = len(conditions) # number of conditions
        n_trials = len(cropped_epochs) # number of trials
        n_samples = cropped_epochs.get_data().shape[2]
        n_channels = cropped_epochs.get_data().shape[1]
        min_time = cropped_epochs.times[0]*1000   # First time point in milli seconds
        max_time = cropped_epochs.times[-1]*1000    # Last time point in milli seconds


        print("=====================================================")
        print("=====================================================")
        print("=====================================================")

        print(f" Condition lists: {conditions}")
        print(f" Total trials: {n_trials}")
        print(f" Time points: {n_samples}")
        print(f" Time window (ms): {min_time} - {max_time}")


        # check GPU availability
        cuda = torch.cuda.is_available()  # Check if a GPU is available
        device = "cuda" if cuda else "cpu"  # Use "cuda" if available, otherwise fallback to "cpu"

        if cuda:
            torch.backends.cudnn.benchmark = True  # Enable cuDNN auto-tuning for performance


        # Set random seeds
        # PURPOSE: reproducibility across runs, random initializations (e.g., model weights) yield consistant results
        seed = 20200220
        set_random_seeds(seed=seed, cuda=cuda)


        # dictionaries for decoding accuracy
        pairwise_decoding_accuracies = {}
        # dictionaries for decoding accuracy of standard deviation
        pairwise_decoding_accuracies_std = {}
        # dictionaries for estimators
        pairwise_decoding_estimators = {}



        for i in range(n_conditions):
            for j in range(i + 1, n_conditions):

                cond1 = conditions[i]
                cond2 = conditions[j]

                filtered_epochs = cropped_epochs[cond1,cond2]

                # get EEG data
                X = filtered_epochs.get_data(picks=mne.pick_types(filtered_epochs.info, eeg=True, eog=False, exclude='bads'))

                # get labels (=numerosity)
                unique_labels = np.unique(filtered_epochs.events[:,-1])
                label_0, label_1 = unique_labels[0], unique_labels[1]  # Assign the first label to 0 and the second to 1

                y = np.where(filtered_epochs.events[:, -1] == label_0, 0, 1)

                counts = np.bincount(y)
                print(f"{cond1} vs {cond2}")
                print(f"{counts[0]} vs. {counts[1]} trials")

                skfold = StratifiedKFold(n_splits=EEGNET_CV, shuffle=True, random_state=23)

                # exp. moving std. for each trial
                for s in range(X.shape[0]):
                    X[s,:,:] = exponential_moving_standardize(X[s,:,:], factor_new=0.001, init_block_size=None, eps=1e-4)

                class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)


                # create the model
                net = EEGClassifier(
                    "EEGNetv4", 
                                module__n_chans=n_channels, # Number of EEG channels
                                module__n_outputs=2,               # Number of outputs of the model. This is the number of classes in the case of classification.
                                module__n_times=n_samples,         # Number of time samples of the input window.
                                module__final_conv_length='auto',  # Length of the final convolution layer. If "auto", it is set based on the n_times.
                                module__pool_mode='mean',          # Pooling method to use in pooling layers
                                module__F1=8,                      # Number of temporal filters in the first convolutional layer.
                                module__D=2,                       # Depth multiplier for the depthwise convolution.
                                module__F2=16,                     # Number of pointwise filters in the separable convolution. Usually set to ``F1 * D``.
                                module__kernel_length=64,         # Length of the temporal convolution kernel. Usally sampling rate / 2 = 500/2 = 250
                                module__third_kernel_size=(8, 4), 
                                module__drop_prob=0.25,            # Dropout probability after the second conv block and before the last layer. 0.5 for within-subject classification, 0.25 in cross-subject clasification
                                module__chs_info=None,             # (list of dict) – Information about each individual EEG channel. This should be filled with info["chs"]. Refer to mne.Info for more details.
                                module__input_window_seconds=None, # Length of the input window in seconds.
                                module__sfreq=SFREQ,
                                max_epochs=EEGNET_MAX_EPOCHS,
                                batch_size=EEGNET_BATCH_SIZE,
                                train_split=None,
                                criterion__weight=torch.Tensor(class_weights).to('cpu'), # class weight
                )
                
                cvs = cross_validate(net, 
                                    X, 
                                    y, 
                                    scoring="balanced_accuracy", # for balanced classes, this corresponds to accuracy,
                                    # chance level might be 0 (adjusted = False), or 0.X (adjusted = True)
                                    cv=skfold, 
                                    n_jobs=-1, # only 1 to avoid overload in parallel jobs, in non-par jobs it could be -1
                                    return_estimator=True, # if you need the model to estimate on another test set
                                    return_train_score=False,
                                    )
                            
                pairwise_decoding_accuracies[(cond1,cond2)] = np.mean(cvs['test_score'])
                pairwise_decoding_accuracies_std[(cond1,cond2)] = np.std(cvs['test_score'])
                pairwise_decoding_estimators[(cond1,cond2)] = cvs['estimator']



        # ~~~~~~~~~~~~~~~~ Save the decoding accuracy
        save_folder = f"{COMPUTE_DIR}/data/{group}/processed/{modality}/{subject}"

        filename = f"EEGNet_accuracy_pairwise_{target_condition}_timewindow350.pkl"
        save_path = os.path.join(save_folder, filename) #  a pickle file
        with open(save_path, "wb") as f:
            pickle.dump(pairwise_decoding_accuracies, f)
        print(f"{subject}: saved in {filename}")
        # ~~~~~~~~~~~~~~~~ Save the decoding accuracy ~~~~~~~~~~~~~~~~

        # ~~~~~~~~~~~~~~~~ Save the decoding accuracy standard deviation
        filename = f"EEGNet_std_pairwise_{target_condition}_timewindow350.pkl"
        save_path = os.path.join(save_folder, filename) #  a pickle file
        with open(save_path, "wb") as f:
            pickle.dump(pairwise_decoding_accuracies_std, f)
        print(f"{subject}: saved in {filename}")
        # ~~~~~~~~~~~~~~~~ Save the decoding accuracy standard deviation ~~~~~~~~~~~~~~~~


        # ~~~~~~~~~~~~~~~~ Save the EEGNet estimation
        filename = f"EEGNet_estimator_pairwise_{target_condition}_timewindow350.pkl"
        save_path = os.path.join(save_folder, filename) #  a pickle file
        with open(save_path, "wb") as f:
            pickle.dump(pairwise_decoding_estimators, f)
        print(f"{subject}: saved in {filename}")
        # ~~~~~~~~~~~~~~~~ Save the EEGNet estimation ~~~~~~~~~~~~~~~~
